##Uploading The Text Files 

In [10]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving nijab_news (1).txt to nijab_news (1) (1).txt
User uploaded file "nijab_news (1).txt" with length 15635 bytes


In [11]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving out (1).txt to out (1) (1).txt
User uploaded file "out (1).txt" with length 222478 bytes


In [12]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving tdd.py to tdd (1).py
User uploaded file "tdd.py" with length 1773 bytes


##Uploading Attention Decoder Python File

In [13]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving attention_decoder.py to attention_decoder (1).py
User uploaded file "attention_decoder.py" with length 14836 bytes


In [14]:
import string

# load doc into memory
def load_doc(filename):
  # open the file as read only
  with open('nijab_news (1).txt', encoding='utf-8', errors='ignore') as f:
    line = f.read()
    return line
 
# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)####
	tokens = [w.translate(table) for w in tokens]######Z
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens
 
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
 
# load document
in_filename = 'nijab_news (1).txt'
doc = load_doc(in_filename)
print(doc[:200])
 
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))####
 
# organize into sequences of tokens
length = 200 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))
 
# save sequences to file
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)

Najib Razak, the former Malaysian prime minister, has been arrested for his alleged role in the 1MDB scandal, in which billions were embezzled from a government fund.
 
Najib, who was toppled from pow
['najib', 'razak', 'the', 'former', 'malaysian', 'prime', 'minister', 'has', 'been', 'arrested', 'for', 'his', 'alleged', 'role', 'in', 'the', 'scandal', 'in', 'which', 'billions', 'were', 'embezzled', 'from', 'a', 'government', 'fund', 'najib', 'who', 'was', 'toppled', 'from', 'power', 'in', 'may', 'was', 'arrested', 'in', 'his', 'kuala', 'lumpur', 'home', 'on', 'tuesday', 'afternoon', 'and', 'will', 'be', 'charged', 'in', 'court', 'in', 'the', 'malaysian', 'capital', 'on', 'wednesday', 'morning', 'according', 'to', 'a', 'statement', 'from', 'the', 'team', 'investigating', 'the', 'former', 'prime', 'minster', 'was', 'picked', 'up', 'from', 'his', 'property', 'using', 'three', 'unmarked', 'cars', 'and', 'taken', 'to', 'the', 'malaysian', 'anticorruption', 'agency', 'macc', 'headquarters',

In [0]:
from google.colab import files
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from attention_decoder import AttentionDecoder


# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')######

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)####
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1####

# separate into input and output###
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [16]:
print ("X-shape is " + str(X.shape))
print ("vocab_size " + str(vocab_size))
print ("seq_length " + str(seq_length))

X-shape is (2255, 200)
vocab_size 786
seq_length 200


In [17]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(AttentionDecoder(100,100))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

inputs shape: (?, ?, 100)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 300)          235800    
_________________________________________________________________
lstm_3 (LSTM)                (None, 200, 100)          160400    
_________________________________________________________________
AttentionDecoder (AttentionD (None, 200, 100)          150600    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 786)               79386     
Total params: 716,686
Trainable params: 716,686
Non-trainable params: 0
____________________________________________

In [18]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=100, epochs=20)
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Epoch 1/20
2255/2255 [==============================] - 276s 123ms/step - loss: 6.5631 - acc: 0.0257
Epoch 2/20
2255/2255 [==============================] - 269s 119ms/step - loss: 5.9699 - acc: 0.0559
Epoch 3/20
2255/2255 [==============================] - 271s 120ms/step - loss: 5.8265 - acc: 0.0572
Epoch 4/20
2255/2255 [==============================] - 272s 120ms/step - loss: 5.7994 - acc: 0.0572
Epoch 5/20
2255/2255 [==============================] - 273s 121ms/step - loss: 5.7974 - acc: 0.0572
Epoch 6/20
2255/2255 [==============================] - 271s 120ms/step - loss: 5.7880 - acc: 0.0572
Epoch 7/20
2255/2255 [==============================] - 271s 120ms/step - loss: 5.7930 - acc: 0.0572
Epoch 8/20
2255/2255 [==============================] - 270s 120ms/step - loss: 5.7870 - acc: 0.0572
Epoch 9/20
2255/2255 [==============================] - 269s 119ms/step - loss: 5.7878 - acc: 0.0572
Epoch 10/20
2255/2255 [==============================] - 271s 120ms/step - loss: 5.7834 - a

In [0]:
files.download('tokenizer.pkl')

In [0]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [21]:
# Upload the file to Drive. See:
#
# https://developers.google.com/drive/v3/reference/files/create
# https://developers.google.com/drive/v3/web/manage-uploads
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'model.h5',
  }
media = MediaFileUpload('model.h5', 
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 19Gl5NUpes687izPJ2JhMzWg163LgpYjU


In [22]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving tokenizer (2).pkl to tokenizer (2).pkl
User uploaded file "tokenizer (2).pkl" with length 34005 bytes


In [23]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Katie_Bouman_BlackHole.txt to Katie_Bouman_BlackHole.txt
User uploaded file "Katie_Bouman_BlackHole.txt" with length 4792 bytes


In [24]:
import string

# load doc into memory
def load_doc(filename):
  # open the file as read only
  with open('Katie_Bouman_BlackHole.txt', encoding='utf-8', errors='ignore') as f:
    line = f.read()
    return line
 
# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)####
	tokens = [w.translate(table) for w in tokens]######Z
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens
 
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
 
# load document
in_filename = 'Katie_Bouman_BlackHole.txt'
doc = load_doc(in_filename)
print(doc[:200])
 
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))####
 
# organize into sequences of tokens
length = 200 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))
 
# save sequences to file
out_filename = 'KT.txt'
save_doc(sequences, out_filename)

The first-ever photo of a black hole released Wednesday has mesmerized the world. Behind the out-of-this-world project is 29-year-old computer scientist Dr. Katie Bouman, who helped form the algorithm
['the', 'firstever', 'photo', 'of', 'a', 'black', 'hole', 'released', 'wednesday', 'has', 'mesmerized', 'the', 'world', 'behind', 'the', 'outofthisworld', 'project', 'is', 'computer', 'scientist', 'dr', 'katie', 'bouman', 'who', 'helped', 'form', 'the', 'algorithm', 'that', 'made', 'the', 'visualization', 'a', 'reality', 'a', 'computer', 'scientist', 'has', 'earned', 'plaudits', 'worldwide', 'for', 'helping', 'develop', 'the', 'algorithm', 'that', 'created', 'the', 'firstever', 'image', 'of', 'a', 'black', 'hole', 'katie', 'bouman', 'led', 'development', 'of', 'a', 'computer', 'program', 'that', 'made', 'the', 'breakthrough', 'image', 'possible', 'the', 'remarkable', 'photo', 'showing', 'a', 'halo', 'of', 'dust', 'and', 'gas', 'million', 'trillion', 'km', 'from', 'earth', 'was', 'released

In [0]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

# load cleaned text sequences
in_filename = 'KT.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
#model = load_model("model (2) (1).h5")

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 200)
print(generated)

##Summary

scientist has earned plaudits worldwide for helping develop the algorithm that created the firstever image of a black hole. Katie bouman led development of a computer program that made the breakthrough image possible. The remarkable photo showing , a halo of dust and gas million trillion km from earth was released on wednesday. The first visualisation of a black hole looks set to revolutionise our understanding of one of the great mysteries of the universe. And the woman whose crucial algorithm helped make it possible is just years old .For dr bouman its creation was the realisation of an endeavour previously thought impossible. Once considered impossible the image captured by the event horizon telescope eht shows a black central core the event horizon surrounded by a lopsided ring of light emitted by particles racing around the black hole at nearly the speed of light a team of over researchers including bouman made it happen. Watching in disbelief as the first image i ever made of a black hole was in the process of being reconstructed the eht postdoctoral fellow wrote in a facebook Post wednesday .Excitedly bracing herself for the groundbreaking moment dr bouman was pictured loading the image on her laptop.







In [31]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Sri_Lanka_Attack.txt to Sri_Lanka_Attack.txt
User uploaded file "Sri_Lanka_Attack.txt" with length 6158 bytes


In [45]:
import string

# load doc into memory
def load_doc(filename):
  # open the file as read only
  with open('Sri_Lanka_Attack.txt', encoding='utf-8', errors='ignore') as f:
    line = f.read()
    return line
 
# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)####
	tokens = [w.translate(table) for w in tokens]######Z
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens
 
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
 
# load document
in_filename = 'Sri_Lanka_Attack.txt'
doc = load_doc(in_filename)
print(doc[:200])
 
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))####
 
# organize into sequences of tokens
length = 200 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))
 
# save sequences to file
out_filename = 'Sri.txt'
save_doc(sequences, out_filename)

Hundreds of police officers swept through Colombo, Sri Lanka, on Thursday, shutting down large parts of the capital city and looking urgently for six suspects  three men and three women  connected to 
['hundreds', 'of', 'police', 'officers', 'swept', 'through', 'colombo', 'sri', 'lanka', 'on', 'thursday', 'shutting', 'down', 'large', 'parts', 'of', 'the', 'capital', 'city', 'and', 'looking', 'urgently', 'for', 'six', 'suspects', 'three', 'men', 'and', 'three', 'women', 'connected', 'to', 'the', 'easter', 'sunday', 'attacks', 'who', 'might', 'be', 'planning', 'a', 'new', 'wave', 'of', 'bloodshed', 'the', 'countrys', 'defense', 'secretary', 'hemasiri', 'fernando', 'resigned', 'after', 'the', 'president', 'blamed', 'him', 'for', 'failing', 'to', 'act', 'on', 'warnings', 'of', 'an', 'impending', 'assault', 'at', 'least', 'days', 'before', 'the', 'attack', 'the', 'sri', 'lankan', 'government', 'had', 'detailed', 'intelligence', 'that', 'islamist', 'extremists', 'were', 'plotting', 'suicide'

In [0]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

# load cleaned text sequences
in_filename = 'Sri.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
#model = load_model("model (2) (1).h5")

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 200)
print(generated)

##Summary

various government agencies was hampering the investigation. Sri Lankan intelligence agencies were still refusing to share foreign intelligence reports with officials in other departments investigating the suicide attacks, the official said .The sri lankan government is bitterly divided ,which may have contributed to the security failures leading up to the bombings. Prime minister ranil wickremesinghe , a rival of president maithripala sirisena, has complained that he was not allowed to attend security meetings before the attacks and was never informed of the warnings .Mr sirisena has tried to deflect intense criticism for the governments failure to prevent the attacks and on wednesday called for he called for the resignations of mr fernando, the defense secretary and pujith jayasundara, the inspector general of the police .Just a few months ago, the authorities arrested and then released several of the suicide bombers, an in january the police found a large cache of weapons ,including explosives, assembled by radical islamists who they say were probably were linked to national thowheeth jamaath. The longanticipated claim of responsibility for the attacks was made by the islamic state is on tuesday night. This could help explain how one local cell based around a single extended family circle of hateful extremists not previously known for terrorism could execute such a massive attack.
